# Creating a basic Cluster Expansion

In [1]:
import numpy as np
import json
from pymatgen import Structure
from smol.cofe import ClusterSubspace, StructureWrangler, ClusterExpansion

In [2]:
prim = Structure.from_file('../data/LiF.cif')
ro = {'Li+':{'Li+':1/3, 'Mn3+':1/3,  'Ti4+':1/3}, 'F-':{'O2-':1.0}}

prim.replace_species(ro)

Loading the DFT data of Li-Mn-Ti-O system

In [3]:

calc_data = []
# calc_indices = []

with open('./data/MnTi_O.json', 'r') as fin: calc_data = json.loads(fin.read())


# print(calc_data)

valid_structs = []
weights = []
for calc_i, calc in enumerate(calc_data):

    struct = Structure.from_dict(calc['s'])
    valid_structs.append((struct, calc['toten']))

print("{}/{} structures map to the lattice".format(len(valid_structs), len(calc_data)))

630/630 structures map to the lattice


### 0) The prim structure
The prim structure defines the **configurational space** for the Cluster Expansion. 
The **configurational space** is defined by the site **compositional spaces** and the crystal symetries of the prim structure.
The occupancy of the sites determine site **compositional spaces**. Sites are **active** if they have compositional degrees of freedom.


Active sites have fractional compositions. Vacancies are allowed in sites where the composition does not sum to one.

0. Is active. The allowed species are: Li+ and vacancies.
1. Is active. The allowed species are: Ni3+ and Ni4+.
2. Is not active. Only O2- is allowed.
3. Is not active. Only O2- is allowed.

In [4]:
print(prim)

Full Formula (Li0.33333333 Ti0.33333333 Mn0.33333333 O1)
Reduced Formula: Li0.33333333Ti0.33333333Mn0.33333333O1
abc   :   2.969848   2.969848   2.969848
angles:  60.000000  60.000000  60.000000
Sites (2)
  #  SP                                   a    b    c
---  ---------------------------------  ---  ---  ---
  0  Li+:0.333, Ti4+:0.333, Mn3+:0.333  0    0    0
  1  O2-                                0.5  0.5  0.5


### 1) The cluster subspace
The `ClusterSubspace` represents all the orbits (groups of equivalent clusters) that will be considered when fitting the cluster expansion. Its main purpose is to compute the **correlations functions** for each included orbit given a structure in the compositional space defined by the prim.

In order to do be able to compute the correlation functions, the given structure must match the prim structure in a "crystallographic" sense but allowing for compositional degrees of freedom in the "active" sites.

A cluster subspace most easily created by providing:
1. The prim structure representing the configurational space.
2. A set of cutoff radii for each size of orbit we want to consider.
3. A type of site basis function to use.

There are more options allowed by the code to fine grain and tune. See other notebooks for advanced use cases.

In [6]:
subspace = ClusterSubspace.from_radii(prim,
                                      radii={2: 7.1, 3: 4, 4:4}, # will include orbits of 2 and 3 sites.
                                      basis='sinusoid',
                                      supercell_size='volume')

# supercell_size specifies the method to determine the supercell size
# when trying to match a structure.
# (See pymatgen.structure_mathcer.StructureMatcher for more info)

# subspace.add_external_term(EwaldTerm(eta=None))

print(subspace) # single site and empty orbits are always included.

ClusterBasis: [Prim Composition] Li+0.33333333 Mn3+0.33333333 Ti4+0.33333333 O2-1
    [Size] 0
      [Orbit] id: 0  orderings: 1
    [Size] 1
      [Orbit] id: 1  orderings: 2   multiplicity: 1    no. symops: 48  
              [Base Cluster] Radius: 0.0   Centroid: [0. 0. 0.]         Points: [[0. 0. 0.]]         
    [Size] 2
      [Orbit] id: 2  orderings: 3   multiplicity: 6    no. symops: 8   
              [Base Cluster] Radius: 2.97  Centroid: [0.  0.  0.5]      Points: [[0. 0. 1.]  [0. 0. 0.]]                  
      [Orbit] id: 3  orderings: 3   multiplicity: 3    no. symops: 16  
              [Base Cluster] Radius: 4.2   Centroid: [0.5 0.5 0.5]      Points: [[1. 1. 0.]  [0. 0. 1.]]                  
      [Orbit] id: 4  orderings: 3   multiplicity: 12   no. symops: 4   
              [Base Cluster] Radius: 5.14  Centroid: [0.  0.5 0.5]      Points: [[0. 1. 1.]  [0. 0. 0.]]                  
      [Orbit] id: 5  orderings: 3   multiplicity: 6    no. symops: 8   
              

#### 1.1) Computing a correlation vector.
A correlation vector for a specific structure (represents the feature vector) used to train and predict target values.

In [7]:
structure = Structure.from_dict(calc_data[0]['s'])
corr = subspace.corr_from_structure(structure)

print(f'The correlation vector for a structure with'
      f' composition {structure.composition} is: '
      f'\n{corr}')

The correlation vector for a structure with composition Li+16 Mn3+16 O2-32 is: 
[ 1.         -0.25        0.4330127  -0.0078125  -0.14884812  0.1640625
  0.109375   -0.08118988  0.203125    0.0859375  -0.09472153  0.1953125
  0.0625     -0.10825318  0.1875     -0.0078125  -0.14884812  0.1640625
  0.05029297  0.02452611 -0.07177734  0.05328086  0.00390625  0.03129193
  0.03222656 -0.02283465  0.01757812]


### 2) The structure wrangler
The `StructureWrangler` is a class that will is used to create and organize the data that will be used to train (and possibly test) the cluster expansion. It makes sure that all the supplied structures appropriately match the prim structure, and obtains the necessary information to correctly normalize target properties (such as energy) necessary for training.

Matching relaxed structures can be a tricky problem, especially for ionic systems with vacancies. See the notebook on structure matching for tips on how to tweak parameters.

In [8]:
wrangler = StructureWrangler(subspace)

# you can add any number of properties and name them
# whatever you want. You should use something descriptive.
# In this case we'll call it 'total_energy'.
for ii, valid in enumerate(valid_structs):
    print("processed: {}/{}".format(ii, len(valid_structs)))
    structure = valid[0]
    wrangler.add_data(structure,
                      properties={'total_energy': valid[1]},
                      verbose=True)
# The verbose flag will print structures that fail to match.

print(f'\nTotal structures that match {wrangler.num_structures}/{len(valid_structs)}')

processed: 0/630
processed: 1/630
processed: 2/630
processed: 3/630
processed: 4/630
processed: 5/630
processed: 6/630
processed: 7/630
processed: 8/630
processed: 9/630
processed: 10/630
processed: 11/630
processed: 12/630
processed: 13/630
processed: 14/630
processed: 15/630
processed: 16/630
processed: 17/630
processed: 18/630
processed: 19/630
processed: 20/630
processed: 21/630
processed: 22/630
processed: 23/630
processed: 24/630
processed: 25/630
processed: 26/630
processed: 27/630
processed: 28/630
processed: 29/630
processed: 30/630
processed: 31/630
processed: 32/630
processed: 33/630
processed: 34/630
processed: 35/630
processed: 36/630
processed: 37/630
processed: 38/630
processed: 39/630
processed: 40/630
processed: 41/630
processed: 42/630
processed: 43/630
processed: 44/630
processed: 45/630
processed: 46/630
processed: 47/630
processed: 48/630
processed: 49/630
processed: 50/630
processed: 51/630
processed: 52/630
processed: 53/630
processed: 54/630
processed: 55/630
pr

processed: 440/630
processed: 441/630
processed: 442/630
processed: 443/630
processed: 444/630
processed: 445/630
processed: 446/630
processed: 447/630
processed: 448/630
processed: 449/630
processed: 450/630
processed: 451/630
processed: 452/630
processed: 453/630
processed: 454/630
processed: 455/630
processed: 456/630
processed: 457/630
processed: 458/630
processed: 459/630
processed: 460/630
processed: 461/630
processed: 462/630
processed: 463/630
processed: 464/630
processed: 465/630
processed: 466/630
processed: 467/630
processed: 468/630
processed: 469/630
processed: 470/630
processed: 471/630
processed: 472/630
processed: 473/630
processed: 474/630
processed: 475/630
processed: 476/630
processed: 477/630
processed: 478/630
processed: 479/630
processed: 480/630
processed: 481/630
processed: 482/630
processed: 483/630
processed: 484/630
processed: 485/630
processed: 486/630
processed: 487/630
processed: 488/630
processed: 489/630
processed: 490/630
processed: 491/630
processed: 4

In [9]:
wrangler.get_property_vector('total_energy')

array([-13.11874117, -13.12903802, -13.17137952, -13.08021449,
       -13.16362834, -13.10222821, -13.08156161, -13.15720459,
       -13.06331364, -13.12062745, -13.17905239, -13.12440631,
       -13.17411988, -13.20385514, -13.10332586, -13.12882567,
       -13.13764792, -13.14550215, -13.1109768 , -13.13433355,
       -13.08290429, -13.15571512, -13.12275268, -13.15715223,
       -13.09132904, -13.08556145, -13.03954859, -13.12791847,
       -13.12750551, -13.08865104, -13.05968182, -13.13590795,
       -13.1004459 , -13.17690217, -13.14593605, -13.11400736,
       -13.12124423, -13.14953541, -13.12336135, -13.13870288,
       -13.14821189, -13.02921938, -13.10899456, -13.01145925,
       -13.02529022, -13.09379601, -13.13166383, -13.06279359,
       -13.11696813, -13.13820625, -13.07975418, -12.97458318,
       -13.1316143 , -13.11638331, -13.15155213, -13.0794376 ,
       -13.13064402, -13.15555385, -13.12331726, -13.10800658,
       -13.01404902, -13.00900896, -13.09916021, -13.10

## 3) Training

Training a cluster expansion is one of the most critical steps. This is how you get **effective cluster interactions (ECI's)**. To do so you need an estimator class that implements some form of regression model. In this case we will use simple least squares regression using the `LinearRegression` estimator from `scikit-learn`.

In `smol` the coefficients from the fit are not exactly the ECI's but the ECI times the multiplicity of their orbit.

Currently we also have the old l1regs estimator from pyabinitio in `smol.learn` as `WDRLasso`, but it will likely be deprecated and removed at some point...

In [10]:
wrangler.feature_matrix

array([[ 1.        , -0.25      ,  0.4330127 , ...,  0.03222656,
        -0.02283465,  0.01757812],
       [ 1.        , -0.25      ,  0.4330127 , ...,  0.03125   ,
        -0.01776029,  0.02636719],
       [ 1.        , -0.25      ,  0.4330127 , ...,  0.0703125 ,
        -0.02029747,  0.        ],
       ...,
       [ 1.        , -0.4375    , -0.10825318, ..., -0.01041667,
         0.        ,  0.        ],
       [ 1.        , -0.4375    , -0.10825318, ..., -0.04166667,
         0.        ,  0.        ],
       [ 1.        , -0.4375    , -0.10825318, ...,  0.02083333,
         0.        ,  0.        ]])

In [11]:
from sklearn.linear_model import LinearRegression
# Set fit_intercept to False because we already do this using
# the empty cluster.
estimator = LinearRegression(fit_intercept=False)
estimator.fit(wrangler.feature_matrix,
              wrangler.get_property_vector('total_energy'))
coefs = estimator.coef_

#### 3.1) Check the quality of the fit
There are many ways to evaluate the quality of a fit. The simplest involve stadard training set prediction error metrics. But when evaluating a CE more seriously we need to consider further metrics and how the CE will be used.
Here we will just look at in sample mean squared error and max error.

In [12]:
from sklearn.metrics import mean_squared_error, max_error

train_predictions = np.dot(wrangler.feature_matrix, coefs)

rmse = mean_squared_error(wrangler.get_property_vector('total_energy'),
                          train_predictions, squared=False)
maxer = max_error(wrangler.get_property_vector('total_energy'),
                  train_predictions)

print(f'RMSE {rmse} eV/prim')
print(f'MAX {maxer} eV/prim')

RMSE 0.013720944562109929 eV/prim
MAX 0.10238344490559825 eV/prim


### 4) The cluster expansion
Now we can use the above work to create the `ClusterExpansion`. The cluster expansion can be used to predict the fitted property for new structures, either for testing quality or for simulations such as in Monte Carlo.
Note that when using the `predict` function, the cluster expansion will have to match the given structure if it has not seen it before.

In [13]:
expansion = ClusterExpansion(subspace,
                             coefficients=coefs,
                             feature_matrix=wrangler.feature_matrix)

structure = np.random.choice(wrangler.structures)
prediction = expansion.predict(structure, normalize=True)

print(f'The predicted energy for a structure with composition '
      f'{structure.composition} is {prediction} eV/prim.\n')
print(f'The fitted coefficients are:\n{expansion.coefs}\n')
print(f'The effective cluster interactions are:\n{expansion.eci}\n')
print(expansion)

The predicted energy for a structure with composition Li+24 Ti4+12 O2-36 is -13.987300872802734 eV/prim.

The fitted coefficients are:
[-1.35544505e+11 -3.38861262e+11  1.17384984e+11  7.93254683e-01
 -5.29937743e-01  1.55309988e-01  4.38745099e-01 -2.09149650e-01
  5.71362778e-02  1.99013562e-01  1.62974586e-01 -1.10869341e-01
  1.44048300e-01  6.10350667e-02 -3.39717283e-02  2.38907852e-01
 -1.30464459e-01 -1.23158891e-02 -2.50362531e-01 -1.03629705e-01
  4.50214013e-01  7.77798931e-02 -6.63667219e-02 -1.05671815e-01
  2.82000038e-01  1.38565502e-01 -1.30884770e-01]

The effective cluster interactions are:
[-1.35544505e+11 -3.38861262e+11  1.17384984e+11  1.32209114e-01
 -8.83229572e-02  2.58849980e-02  1.46248366e-01 -6.97165500e-02
  1.90454259e-02  1.65844635e-02  1.35812155e-02 -9.23911179e-03
  2.40080499e-02  1.01725111e-02 -5.66195471e-03  1.99089877e-02
 -1.08720382e-02 -1.02632409e-03 -3.12953163e-02 -1.29537132e-02
  5.62767516e-02  9.72248664e-03 -3.31833610e-02 -5.2835907

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """


### 5) Saving your work
All core classes in `smol` are `MSONables` and so can be saved using their `as_dict` methods or better yet with `monty.serialization.dumpfn`.

Currently there is also a convenience function in `smol` that will nicely save all of your work for you in a standardized way. Work saved with the `save_work` function is saved as a dictionary with standardized names for the classes. Since a work flow should only contain 1 of each core classes the function will complain if you give it two of the same class (i.e. two wranglers)

In [16]:
from smol.io import load_work, save_work

file_path = 'data/LMTO_sinusoid.mson'
# we can save the subspace as well, but since both the wrangler
# and the expansion have it, there is no need to do so.
save_work(file_path, wrangler, expansion)

#### 5.1) Loading previously saved work

In [17]:
from smol.io import load_work, save_work

work = load_work(file_path)
for name, obj in work.items():
    print(f'{name}: {type(obj)}\n')

StructureWrangler: <class 'smol.cofe.wrangler.StructureWrangler'>

ClusterExpansion: <class 'smol.cofe.expansion.ClusterExpansion'>

